In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.datasets import load_iris

In [2]:
x,y = load_iris(return_X_y=True)
x.shape, y.shape, set(y)

((150, 4), (150,), {0, 1, 2})

총 150개의 데이터가 있으며, {0, 1, 2}라는 3개의 클래스가 있음을 볼 수 있다.

In [28]:
class MyModel(keras.Model):
    
    '''
    __init__ 생성자 함수
    optimizer: SGD(확률적 경사하강법) learning_rate = 0.01
    하나의 dense 층을 사용
    3개의 클래스이므로 units=3
    activation function: softmax
    '''
    def __init__(self):
        super(MyModel, self).__init__() # 상속한 클래스의 생성자 호출
        self.opt = tf.optimizers.SGD(learning_rate=0.01)
        self.dense = keras.layers.Dense(units=3, activation=keras.activations.softmax)
    
    '''
    입력값을 받아
    생성자에서 만든 layer를 불러와 학습을 진행한다.
    '''
    def call(self, x):
        h = self.dense(x)
        return h
    
    def get_loss(self, y, h):
        cross_entropy = - (y * tf.math.log(h) + (1-y) * tf.math.log(1-h))
        loss = tf.reduce_mean(cross_entropy)
        return loss
    
    '''
    정확도 측정 함수
    h 3개의 확률: (m,3), y: (m)
    '''
    def get_accuracy(self, y, h):
        predict = tf.argmax(h, -1)
        self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32)) 
        # True > 1, False > 0
    
    '''
    1차원이던 y를 one hot 인코딩을 통해 (m,3) 차원으로 바꿔준다.
    epoch 만큼 경사하강을 진행한다. 
    경사하강을 진행하는 동안 경사 기록 장치를 통해 변경되는 값들을 기록한다.
    또한 경사 값을 계산하고 가중치에서 경사를 빼는 과정을 수행한다.
    
    x : (m, 4), y : (m)
    '''
    def train(self, x, y, epoch=1):
        y_hot = tf.one_hot(y, depth=3, axis=-1)
        y_hot = tf.cast(y_hot, tf.float32)
        for i in range(epoch):
            with tf.GradientTape() as tape:
                h = self.call(x)
                loss = self.get_loss(y_hot, h)
            grads = tape.gradient(loss, self.trainable_variables) # 경사 계산
            self.opt.apply_gradients(zip(grads, self.trainable_variables)) # 가중치에서 경사 빼기
            self.get_accuracy(y, h)
            print(f'{i}/{epoch} loss: {loss} acc: {self.acc}')

In [29]:
model = MyModel()

In [30]:
model.train(x, y, 100)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0/100 loss: 1.311767816543579 acc: 0.3333333432674408
1/100 loss: 1.2121511697769165 acc: 0.3333333432674408
2/100 loss: 1.120406985282898 acc: 0.3333333432674408
3/100 loss: 1.0366894006729126 acc: 0.3333333432674408
4/100 loss: 0.9609566926956177 acc: 0.3333333432674408
5/100 loss: 0.8930127620697021 acc: 0.3400000035762787
6/100 loss: 0.8325497508049011 acc: 0.35333332419395447
7/100 loss: 0.7791854739189148 acc: 0.36000001430511475
8/100 loss: 0.7324867844581604 acc: 0.36666667461395264
9/100 loss: 

---
## 10에서 진행한 digit을 keras로 진행

In [57]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
%matplotlib inline

10개의 클래스가 존재 

In [58]:
x, y = load_digits(return_X_y=True)
x.shape, y.shape, set(y)

((1797, 64), (1797,), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

훈련 데이터 / 테스트 데이터 나누기

In [59]:
split_index = len(x) // 2
x_train, x_test = x[:split_index], x[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

간단한 모델

In [60]:
simple_model = keras.Sequential()
simple_model.add(keras.layers.Flatten(input_shape=(8*8,)))
simple_model.add(keras.layers.Dense(128, activation='relu'))
simple_model.add(keras.layers.Dense(128, activation='relu'))
simple_model.add(keras.layers.Dense(10, activation=keras.layers.Softmax()))

simple_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

In [61]:
simple_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_28 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_29 (Dense)             (None, 10)                1290      
Total params: 26,122
Trainable params: 26,122
Non-trainable params: 0
_________________________________________________________________


In [62]:
simple_model.fit(x_train, y_train, epochs=7)

simple_model.evaluate(x_test, y_test, verbose=2)

Train on 898 samples
Epoch 1/7
898/898 [==============================] - 0s 324us/sample - loss: 1.5744 - accuracy: 0.5612
Epoch 2/7
898/898 [==============================] - 0s 41us/sample - loss: 0.2296 - accuracy: 0.9354
Epoch 3/7
898/898 [==============================] - 0s 42us/sample - loss: 0.0970 - accuracy: 0.9800
Epoch 4/7
898/898 [==============================] - 0s 40us/sample - loss: 0.0605 - accuracy: 0.9889
Epoch 5/7
898/898 [==============================] - 0s 41us/sample - loss: 0.0359 - accuracy: 0.9955
Epoch 6/7
898/898 [==============================] - 0s 40us/sample - loss: 0.0272 - accuracy: 0.9967
Epoch 7/7
898/898 [==============================] - 0s 41us/sample - loss: 0.0165 - accuracy: 1.0000
899/1 - 0s - loss: 0.1740 - accuracy: 0.9088


[0.32392970074146354, 0.90878755]

In [145]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.opt = tf.optimizers.Adam(learning_rate=0.0002)
        self.conv1 = keras.layers.Conv2D(64, 3, padding='same', input_shape=(8,8,1), activation='relu')
        self.max1 = keras.layers.MaxPool2D((2,2), padding='same')
        self.conv2 = keras.layers.Conv2D(128, 3, input_shape=(8,8,1), activation='relu')
        self.max2 = keras.layers.MaxPool2D((2,2), padding='same')
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(128, activation='relu')
        self.d2 = keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
#         x = self.max1(x)
#         x = self.conv2(x)
#         x = self.max2(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
    def get_loss(self, y, h):
        cross_entropy = - (y * tf.math.log(h) + (1-y) * tf.math.log(1-h))
        loss = tf.reduce_mean(cross_entropy)
        return loss
    
    def get_accuracy(self, y, h):
        predict = tf.argmax(h, -1)
        self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32))
        
    def train(self, x, y, epoch=1):
        y_hot = tf.one_hot(y, depth=10, axis=-1)
        y_hot = tf.cast(y_hot, tf.float32)
        for i in range(epoch):
            with tf.GradientTape() as tape:
                h = self.call(x)
                loss = self.get_loss(y_hot, h)
            grads = tape.gradient(loss, self.trainable_variables) # 경사 계산
            self.opt.apply_gradients(zip(grads, self.trainable_variables)) # 가중치에서 경사 빼기
            self.get_accuracy(y, h)
            print(f'{i}/{epoch} loss: {loss} acc: {self.acc}') 

In [146]:
model = MyModel()

In [147]:
train_x = x_train.reshape((-1,8,8,1))

In [148]:
model.train(train_x, y_train, 100)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0/100 loss: 0.3816850781440735 acc: 0.15033407509326935
1/100 loss: 0.32019829750061035 acc: 0.2327394187450409
2/100 loss: 0.278273344039917 acc: 0.390868604183197
3/100 loss: 0.24241940677165985 acc: 0.5534521341323853
4/100 loss: 0.20956459641456604 acc: 0.685968816280365
5/100 loss: 0.17866134643554688 acc: 0.7616926431655884
6/100 loss: 0.15194232761859894 acc: 0.8151447772979736
7/100 loss: 0.12899085879325867 acc: 0.858574628829956
8/100 loss: 0.11048834025859833 acc: 0.8763920068740845
9/100 los